In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os, tqdm
import numpy as np
import xarray as xr
import torch

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../mre_ai')
#import mre_ai.mre_datasets

torch.cuda.is_available()

Using backend: pytorch



True

## Process images for a single patient

- load SITK images from NIFIT files
- perform filtering/inpainting on wave image
- register images wrt anatomical image
- load segmentation model
- perform image segmentation
- convert images to xarray Dataset

In [26]:
%autoreload
patient = mre_pinn.data.Patient(
    nifti_dir='/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI',
    patient_id='0006',
    xarray_dir='../data/NAFLD/'
)

## Process images for a patient cohort

In [92]:
%autoreload
cohort = mre_pinn.data.PatientCohort(
    nifti_dirs='/ocean/projects/asc170022p/shared/Data/MRE/*/NIFTI',
    patient_ids=['0006', '001', '1081'],
    xarray_dir='../data/NAFLD/'
)
len(cohort)

/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0822 is missing sequences {'mre_raw', 'mre', 't2', 'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0830 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0838 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0846 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0850 is missing sequences {'mre_raw', 'mre', 't2', 'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0854 is missing sequences {'t1_pre_fat', 't1_pre_water', 't2', 'wave', 't1_pre_in', 't1_pre_out'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0859 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIFTI/0860 is missing sequences {'wave'}
/ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_10-09-19/NIF

3

In [93]:
%%time
cohort.load_images()

  0%|          | 0/3 [00:00<?, ?it/s]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/mre.nii
 33%|███▎      | 1/3 [00:00<00:00,  2.35it/s]Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/001/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/001/t1_pre_wa

In [94]:
%%time
m = cohort.metadata
m

CPU times: user 50.2 ms, sys: 1.1 ms, total: 51.3 ms
Wall time: 50.3 ms


size    spacing      origin       limit  \
patient_id sequence     dimension                                            
0006       t1_pre_in    0           512   0.703100 -204.130005  155.154108   
                        1           512   0.703100 -208.591995  150.692118   
                        2            88   2.300003 -142.013000   58.087265   
           t1_pre_water 0           512   0.703100 -204.130005  155.154108   
                        1           512   0.703100 -208.591995  150.692118   
...                                 ...        ...         ...         ...   
1081       wave         0           512   1.000000    0.000000  511.000000   
                        1           512   1.000000    0.000000  511.000000   
           mre          0           256   1.640600 -204.130005  214.222986   
                        1           256   1.640600 -242.070999  176.281992   
                        2             1  11.000000  111.867996  111.867996   

                                       center      extent  
patient_id sequence     dimension                          
0006       t1_pre_in    0          -24.487948  359.987213  
                        1          -28.949939  359.987213  
                        2          -41.962868  202.400269  
           t1_pre_water 0          -24.487948  359.987213  
                        1          -28.949939  359.987213  
...                                       ...         ...  
1081       wave         0          255.500000  512.000000  
                        1          255.500000  512.000000  
           mre          0            5.046491  419.993591  
                        1          -32.894503  419.993591  
                        2          111.867996   11.000000  

[71 rows x 6 columns]

In [99]:
(m.reset_index().groupby(['patient_id', 'sequence']).dimension.count() == 3) \
    .reset_index().groupby(['patient_id']).dimension.all()

patient_id
0006     True
001      True
1081    False
Name: dimension, dtype: bool